In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import tensorflow as tf
from typing import Tuple

Now that you've learned about tensors, variables, and automatic differentiation, you're ready to learn how to define a neural network from scratch using lower-level TensorFlow operations. It's important to understand these foundational concepts because they give you the flexibility to customize your neural networks in any way you like.

As a reminder, here's the structure of the neural network we defined using Keras:

![Basic neural network](notebooks/images/1-fashion-nn.png)

And here's the same neural network, this time showing the $W$ and $b$ parameters we want to learn during training: 

![Basic neural network with details](notebooks/images/2-basic-nn-with-details.png)

Notice that our neural network is composed of three `Dense` layers, and that the first two of those are followed by a `ReLU` ("Rectified Linear Unit") activation function. In Keras, we can build our model by simply initializing a sequence of three `Dense` objects with the appropriate activation functions. If we don't want to use Keras, we need to understand the operations performed by a `Dense` layer a bit better and replicate them. It turns out that a `Dense` layer is just about the simplest type of layer we can use, so it's not that hard to understand and replicate. A `Dense` layer takes as input a $W$ parameter, a $b$ parameter, and data $x$ that comes from the previous layer, and produces the following output:

$$
z = Wx + b
$$

The output of the first two `Dense` layers is then passed as input to a `ReLU` non-linear function in the following way:

$$
a = ReLU(z)
$$

Let's now see how we can represent these concepts using TensorFlow code.

In [ ]:
class NeuralNetwork(tf.keras.Model):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    initializer = tf.keras.initializers.GlorotUniform()
    self.W1 = tf.Variable(initializer(shape=(784, 512)))
    self.b1 = tf.Variable(tf.zeros(shape=(512,)))
    self.W2 = tf.Variable(initializer(shape=(512, 512)))
    self.b2 = tf.Variable(tf.zeros(shape=(512)))
    self.W3 = tf.Variable(initializer(shape=(512, 10)))
    self.b3 = tf.Variable(tf.zeros(shape=(10,)))

  def call(self, x: tf.Tensor) -> tf.Tensor:
    x = tf.reshape(x, [-1, 784])
    x = tf.matmul(x, self.W1) + self.b1
    x = tf.nn.relu(x)
    x = tf.matmul(x, self.W2) + self.b2
    x = tf.nn.relu(x)
    x = tf.matmul(x, self.W3) + self.b3
    return x

Notice that here we're explicitly instantiating parameters $W$ and $b$ using TensorFlow `Variables` (rather than `Tensors`) because their values change during training. Notice also how we initialize their values &mdash; the parameters $b$ are typically initialized to zero, and the parameters $W$ are typically initialized to small random values. There are many methods for parameter initialization, and the choice of technique can have a big effect on how well the neural network trains.

Other than that, you can see that the additions, multiplications, and `ReLU` calls we discussed earlier are reflected in the code as you might expect.